<img style="float: left;padding: 1.3em" src="https://indico.in2p3.fr/event/18313/logo-786578160.png">  

#  Gravitational Wave Open Data Workshop #4


#### Tutorial 1.1: Discovering open data from GW observatories

This notebook describes how to discover what data are available from the [Gravitational-Wave Open Science Center (GWOSC)](https://www.gw-openscience.org).
    
[Click this link to view this tutorial in Google Colaboratory](https://colab.research.google.com/github/gw-odw/odw-2021/blob/master/Tutorials/Day_1/Tuto%201.1%20Discovering%20Open%20Data.ipynb)

## Software installation  (execute only if running on a cloud platform or haven't done the installation yet!)

First, we need to install the software, which we do following the instruction in [Software Setup Instructions](https://github.com/gw-odw/odw-2021/blob/master/setup.md):

In [1]:
# -- Uncomment following line if running in Google Colab
#! pip install -q 'gwosc==0.5.4'

In [1]:
#check the version of the package gwosc you are using
import gwosc
print(gwosc.__version__)

0.5.6


The version you get should be 0.5.4. If it's not, check that you have followed all the steps in [Software Setup Instructions](https://github.com/gw-odw/odw-2021/blob/master/setup.md).

## Querying for event information

The module `gwosc.catalog`  provides tools to search for events in a catalog.

The module `gwosc.datasets` provides tools for searching for datasets, including full run strain data releases.


For example, we can search for events in the [GWTC-1 catalog](https://www.gw-openscience.org/eventapi/html/GWTC-1-confident/), the catalog of all events from the O1 and O2 observing runs. A list of available catalogs can be seen in the [Event Portal](https://gw-openscience.org/eventapi)

In [20]:
from gwosc.datasets import find_datasets
from gwosc import catalog

#-- Print all the GW events from the GWTC-1 catalog 
gwtc1 = catalog.events('GWTC-1-confident')
print('GWTC-1 events:', gwtc1)
print(len(gwtc1))
print("")

#-- Print all the large strain data sets from LIGO/Virgo observing runs
runs = find_datasets(
    detector=None,
    type="run",
    segment=None,
    match=None,
    catalog=None,
    version=None,)
print('Large data sets:', runs)

GWTC-1 events: ['GW150914', 'GW151012', 'GW151226', 'GW170104', 'GW170608', 'GW170729', 'GW170809', 'GW170814', 'GW170817', 'GW170818', 'GW170823']
11

Large data sets: ['BKGW170608_16KHZ_R1', 'O1', 'O1_16KHZ', 'O2_16KHZ_R1', 'O2_4KHZ_R1', 'O3a_16KHZ_R1', 'O3a_4KHZ_R1', 'S5', 'S6', 'oldhistory']


_Attention: Note that the most recent observation runs, e.g. O2, are labeled with names containing the name of the run (e.g. O2), the sampling rate (4 or 16 kHz) and the release version (e.g. R1). This means that for O2 you have two labels 'O2_4KHZ_R1' and 'O2_16KHZ_R1', depending which is the desired sampling rate_

`catalog.events` also accepts a `segment` and `detector` keyword to narrow results based on GPS time and detector:

In [21]:
#-- Detector and segments keywords limit search result
#-- Events for a particular detector
print(catalog.events('GWTC-1-confident', detector="H1", segment=(1164556817, 1187733618)))

['GW170104', 'GW170608', 'GW170729', 'GW170809', 'GW170814', 'GW170817', 'GW170818', 'GW170823']


Using `gwosc.datasets.event_gps`, we can query for the GPS time of a specific event:

In [29]:
from gwosc.datasets import event_gps
gps = event_gps('GW170104')
print(gps)

1167559936.6


<div class="alert alert-info">All of these times are returned in the GPS time system, which counts the number of seconds that have elapsed since the start of the GPS epoch at midnight (00:00) on January 6th 1980. GWOSC provides a <a href="https://www.gw-openscience.org/gps/">GPS time converter</a> you can use to translate into datetime, or you can use <a href="https://gwpy.github.io/docs/stable/time/"><code>gwpy.time</code></a>.</div>

We can query for the GPS time interval for an observing run:

In [24]:
import gwpy.time as time
from gwosc.datasets import run_segment
import numpy as np

for i in ['O2_16KHZ_R1', 'O2_4KHZ_R1']:
    x = run_segment(i)
    print(x)
    print(x[1] - x[0])
    print(list(map(time.from_gps, x)))

(1164556817, 1187733618)
23176801
[datetime.datetime(2016, 11, 30, 16, 0), datetime.datetime(2017, 8, 25, 22, 0)]
(1164556817, 1187733618)
23176801
[datetime.datetime(2016, 11, 30, 16, 0), datetime.datetime(2017, 8, 25, 22, 0)]


To see only the confident events in O1:

In [39]:
O1_events = catalog.events('GWTC-1-confident', segment = run_segment('O1_16KHZ'))
print(O1_events)

['GW150914', 'GW151012', 'GW151226']


## Querying for data files

The `gwosc.locate` module provides a function to find the URLs of data files associated with a given dataset.

For event datasets, one can get the list of URLs using only the event name:

In [41]:
from gwosc.locate import get_event_urls
urls = get_event_urls('GW150914')
print(urls)

['https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5', 'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_4KHZ_R1-1126257415-4096.hdf5']


By default, this function returns all of the files associated with a given event, which isn't particularly helpful. However, we can can filter on any of these by using keyword arguments, for example to get the URL for the 32-second file for the LIGO-Livingston detector:

In [45]:
urls = get_event_urls('GW170817', duration=4096, detector='V1')
print(urls)

['https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW170817/v3/V-V1_GWOSC_4KHZ_R1-1187006835-4096.hdf5']


# Exercises

Now that you've seen examples of how to query for dataset information using the `gwosc` package, please try and complete the following exercies using that interface:

- How many months did O2 last? (Hint: check the output of _find_datasets(type='run')_ to find the correct label to use)
- How many GWTC-1-confident events were detected during O1?
- What file URL contains data for V1 4096 seconds around GW170817?